In [1]:
import json
import pandas as pd

In [33]:
df_acidentes = pd.read_csv("dados/acidentes_naofatais.csv", encoding = "latin", sep = ";")
df_acidentes.head()

,ID,Data do Acidente,Dia do Acidente,Mês do Acidente,Ano do Acidente,Ano/Mês do Acidente,Dia da Semana,Hora do Acidente,Turno,Município,...,Serviço de Atendimento - PRF,Serviço de Atendimento - Radio Patrulha,Servciço de Atendimento - CPTRAN,Tipo de Via,Tipo de Acidente - Atropelamento (Pedestre),Tipo de Acidente - Atropelamento (Animal),Tipo de Acidente - Choque,Tipo de Acidente - Colisão,Tipo de Acidente - Outros tipos de Acidente,Turno + Dia de Semana
0,9533066,2023-09-30,30,SETEMBRO,2023,2023.09,SÁBADO,12:17,TARDE,BIRIGUI,...,NÃO,NÃO,NÃO,Rodovias,0,0,0,0,1,TARDE DE SÁBADO
1,9536362,2023-09-30,30,SETEMBRO,2023,2023.09,SÁBADO,22:54,NOITE,CAMPINAS,...,NÃO,NÃO,NÃO,Vias Municipais,0,0,0,0,0,NOITE DE SÁBADO
2,9534538,2023-09-30,30,SETEMBRO,2023,2023.09,SÁBADO,23:40,NOITE,MOGI GUACU,...,NÃO,NÃO,NÃO,Rodovias,0,0,0,1,0,NOITE DE SÁBADO
3,9533318,2023-09-30,30,SETEMBRO,2023,2023.09,SÁBADO,14:14,TARDE,LIMEIRA,...,NÃO,NÃO,NÃO,Vias Municipais,0,0,0,0,1,TARDE DE SÁBADO
4,9534072,2023-09-30,30,SETEMBRO,2023,2023.09,SÁBADO,19:30,NOITE,SALTO,...,NÃO,SIM,NÃO,Vias Municipais,0,0,0,0,0,NOITE DE SÁBADO


In [39]:
df_lentidao = (
    pd.read_csv("dados/lentidao.csv")
    .replace(
        {"corredor": {"Av": "Avenida", ",": "", "/": " ", "\(": " ", "\)": " ", "\.": " "}}
        , regex = True)
)
df_lentidao.head()

,data,corredor,sentido,expressa,descricao,tamanho,nome_regiao
0,01/01/2022 00:00,Edgar Facó Avenida,Bairro/Centro,NaN,De PAULA FERREIRA até 404 m depois de GIACOMO ...,456,NORTE
1,01/01/2022 00:30,Edgar Facó Avenida,Bairro/Centro,NaN,De PAULA FERREIRA até 404 m depois de GIACOMO ...,456,NORTE
2,01/01/2022 01:00,Edgar Facó Avenida,Bairro/Centro,NaN,De PAULA FERREIRA até 404 m depois de GIACOMO ...,456,NORTE
3,01/01/2022 01:30,Edgar Facó Avenida,Bairro/Centro,NaN,De PAULA FERREIRA até 404 m depois de GIACOMO ...,456,NORTE
4,01/01/2022 02:00,Edgar Facó Avenida,Bairro/Centro,NaN,De PAULA FERREIRA até 404 m depois de GIACOMO ...,456,NORTE


In [34]:
dicio_conversao = json.load(open('cruzamento_infosiga-lentidao.json'))

In [96]:
df = (
pd.DataFrame({"Logradouro": dicio_conversao.keys(), "corredor": dicio_conversao.values()})
.merge(df_lentidao, how = "right", on = "corredor")
.dropna(subset = "Logradouro")
)
df.head()

,Logradouro,corredor,data,sentido,expressa,descricao,tamanho,nome_regiao
48,AVENIDA INTERLAGOS,Interlagos Avenida I,03/01/2022 07:30,Bairro/Centro,NaN,De 408 m antes de ALLYRIO HUGUENEY DE MATTOS O...,1292,SUL
49,AVENIDA INTERLAGOS,Interlagos Avenida I,03/01/2022 08:00,Bairro/Centro,NaN,De 408 m antes de ALLYRIO HUGUENEY DE MATTOS O...,1292,SUL
50,AVENIDA INTERLAGOS,Interlagos Avenida I,03/01/2022 08:30,Bairro/Centro,NaN,De 408 m antes de ALLYRIO HUGUENEY DE MATTOS O...,1292,SUL
51,AVENIDA PROFESSOR ABRAAO DE MORAIS,Abraão de Morais Avenida Prof Imig,03/01/2022 09:00,Santos/São Paulo,NaN,De ALIOMAR BALEEIRO até MATHEUS TORLONI,2080,SUL
52,AVENIDA INTERLAGOS,Interlagos Avenida I,03/01/2022 09:00,Bairro/Centro,NaN,De 408 m antes de ALLYRIO HUGUENEY DE MATTOS O...,1292,SUL


In [87]:
df["data"].str[6:10]

48         2022
49         2022
50         2022
51         2022
52         2022
           ... 
1825235    2015
1825244    2015
1825252    2015
1825261    2015
1825269    2015
Name: data, Length: 109644, dtype: object

In [136]:
(
df
.sort_values(["corredor", "data", "sentido", "tamanho"], ascending = False)
.groupby(["corredor", "data", "sentido"])
.first()
.reset_index()
.assign(ano = lambda _: _["data"].str[6:10])
.query("ano in ['2015', '2016', '2017', '2021', '2022']")
.groupby(["ano", "corredor", "descricao"])
.aggregate({"tamanho": "count"})
.reset_index()
.sort_values(["ano", "corredor", "tamanho"], ascending= False)
.groupby(["ano", "corredor"])
.nth([0,10])
.reset_index()
.sort_values(["corredor", "ano", "tamanho"], ascending= False)
.head(60)
)

,ano,corredor,descricao,tamanho
104,2022,Washington Luis Avenida,De JOAO JULIAO DA COSTA AGUIAR até LINNEU GOME...,1644
105,2022,Washington Luis Avenida,De JOAO JULIAO DA COSTA AGUIAR até OTAVIO TARQ...,31
51,2021,Washington Luis Avenida,De JOAO JULIAO DA COSTA AGUIAR até LINNEU GOME...,456
52,2021,Washington Luis Avenida,De JOAO JULIAO DA COSTA AGUIAR até OTAVIO TARQ...,16
102,2022,Vinte Três R Berta M Guimarães,De PEDROSO até BANDEIRA,1159
103,2022,Vinte Três R Berta M Guimarães,De JOAO JULIAO DA COSTA AGUIAR até REPUBLICA A...,80
49,2021,Vinte Três R Berta M Guimarães,De SAO JOAQUIM até BANDEIRA,428
50,2021,Vinte Três R Berta M Guimarães,De SAO JOAQUIM até TUTOIA,40
100,2022,Salim Farah Maluf Avenida Tatuapé Pte,De ULISSES CRUZ até TATUAPE início,1370
101,2022,Salim Farah Maluf Avenida Tatuapé Pte,De ACRE até MONTE AZUL,35


In [14]:
print(list({key: value for key, value in dicio_conversao.items() if value != None}.keys()))

['AVENIDA VEREADOR JOAO DE LUCA', 'AVENIDA GENERAL ATALIBA LEONEL', 'AVENIDA IBIRAPUERA', 'AVENIDA PROFESSOR ABRAAO DE MORAIS', 'AVENIDA CARLOS CALDEIRA FILHO', 'AVENIDA ENGENHEIRO CAETANO ALVARES', 'AVENIDA CORIFEU DE AZEVEDO MARQUES', 'AVENIDA CELSO GARCIA', 'AVENIDA MORUMBI', 'AVENIDA SANTO AMARO', 'RUA VERGUEIRO', 'AVENIDA INAJAR DE SOUZA', 'AVENIDA JOAO DIAS', 'ACESSO RADIAL LESTE  CENTRO', 'AVENIDA NOVE DE JULHO', 'AVENIDA REBOUCAS', 'AVENIDA BRIGADEIRO LUIS ANTONIO', 'AVENIDA RAIMUNDO PEREIRA DE MAGALHAES', 'AVENIDA SALIM FARAH MALUF', 'AVENIDA ATLANTICA', 'AVENIDA PROFESSOR FRANCISCO MORATO', 'AVENIDA GUARAPIRANGA', 'AVENIDA PROFESSOR LUIZ IGNACIO ANHAIA MELLO', 'AVENIDA VINTE E TRES DE MAIO', 'AVENIDA DO ESTADO', 'AVENIDA DOS BANDEIRANTES', 'AVENIDA INTERLAGOS', 'AVENIDA WASHINGTON LUIS', 'AVENIDA ARICANDUVA', 'ESTRADA DE ITAPECERICA', 'AVENIDA SENADOR TEOTONIO VILELA']
